In [30]:
import uuid
import boto3
import pprint
import botocore
import logging

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)


In [31]:
# global constants
ENDPOINT_URL: str = None


In [32]:
# you want to make sure that install sequence is as follows
# %pip install boto3-1.28.54-py3-none-any.whl
# %pip install botocore-1.31.54-py3-none-any.whl
# %pip install awscli-1.29.54-py3-none-any.whl

# exit out if the Boto3 (Python) SDK versions are not correct
assert boto3.__version__ == "1.28.73"
assert botocore.__version__ == "1.31.73"


In [33]:
input_text:str = "What are the XGBoost versions supported in Amazon SageMaker?" # replace this with a prompt relevant to your agent
agent_id:str = 'J0TEWQNZ89' # note this from the agent console on Bedrock
agent_alias_id:str = 'TSTALIASID' # fixed for draft version of the agent
session_id:str = str(uuid.uuid1()) # random identifier
enable_trace:bool = True


In [34]:
# create an boto3 bedrock agent client
client = boto3.client("bedrock-agent-runtime", endpoint_url=ENDPOINT_URL)
logger.info(client)


[2023-11-02 15:01:55,355] p43792 {691742557.py:3} INFO - <botocore.client.BedrockAgentRuntimeService object at 0x0000029010DFBE50>


In [35]:
# invoke the agent API
response = client.invoke_agent(inputText=input_text,
    agentId=agent_id,
    agentAliasId=agent_alias_id,
    sessionId=session_id,
    enableTrace=enable_trace
)

logger.info(pprint.pprint(response))


[2023-11-02 15:01:55,849] p43792 {4226590062.py:9} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Thu, 02 Nov 2023 19:01:55 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '4a5687bd-79b2-11ee-943b-846a79be0989',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'b76650eb-ee2d-4120-a091-d680ea4c588f'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'b76650eb-ee2d-4120-a091-d680ea4c588f',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x0000029010E0DB90>,
 'contentType': 'application/json',
 'sessionId': '4a5687bd-79b2-11ee-943b-846a79be0989'}


In [36]:
%%time
import json
event_stream = response['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}") 
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)


[2023-11-02 15:02:04,472] p43792 {<timed exec>:11} INFO - {
  "agentId": "J0TEWQNZ89",
  "agentAliasId": "TSTALIASID",
  "sessionId": "4a5687bd-79b2-11ee-943b-846a79be0989",
  "trace": {
    "rationaleTrace": {
      "traceId": "c5e00690-fbdc-4823-a9ac-5ba9ba27c90a-0",
      "text": "Review the \"User Input\", \"Conversation History\", \"Attributes\", \"APIs\" and always think about what to do"
    }
  }
}
[2023-11-02 15:02:19,633] p43792 {<timed exec>:11} INFO - {
  "agentId": "J0TEWQNZ89",
  "agentAliasId": "TSTALIASID",
  "sessionId": "4a5687bd-79b2-11ee-943b-846a79be0989",
  "trace": {
    "invocationInputTrace": {
      "traceId": "c5e00690-fbdc-4823-a9ac-5ba9ba27c90a-0",
      "invocationType": "KNOWLEDGE_BASE",
      "knowledgeBaseLookupInput": {
        "text": "What are the XGBoost versions supported in Amazon SageMaker?",
        "knowledgeBaseId": "HK7XZ6KQYP"
      }
    }
  }
}
[2023-11-02 15:02:19,634] p43792 {<timed exec>:11} INFO - {
  "agentId": "J0TEWQNZ89",
  "agentA

CPU times: total: 15.6 ms
Wall time: 41.9 s
